# 01. Collect Data

## Setting

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [13]:
# url = "https://pcmap.place.naver.com/restaurant/13166754/review/visitor"

url = "https://pcmap.place.naver.com/restaurant/1804344332/review/visitor"

## 1. 순차적 개발

### 페이지 접근

In [14]:
# 크롬창 열기
options = Options()
driver = webdriver.Chrome(options = options)

# 창모드 전체화면으로 크기 늘리기
driver.maximize_window()

# 로딩시간: 페이지 로딩
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))

# 네이버 리뷰 주소 접속하기
driver.get(url)

# 로딩시간: 페이지 로딩
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))

[<selenium.webdriver.remote.webelement.WebElement (session="abfc75c1e5d28f8f20e5069dec498db4", element="f.93FC24F63969894F6BFE5CFC721CF409.d.1DE5C69F5F90ECACA158114ADF7972D2.e.156")>]

In [22]:
driver.get("https://pcmap.place.naver.com/restaurant/13166754/review/visitor")

### ➕ 최신순으로 정렬

* 추천순은 좋은 반응으로 편향될 가능성이 있다고 판단하여 최신순으로 정렬하는 과정을 추가함

In [23]:
driver.find_elements(By.CLASS_NAME, "zMIkw")[-1].click()

### 댓글 더보기

* "더 보기" 버튼을 눌러야 다음 댓글 생성됨
* 리뷰의 끝이 보이면 "더 보기" 버튼은 없어짐 ❌ 에러 발생

In [ ]:
# 해당 클래스가 있는지 없는지 판단하는 함수 
# def is_element(driver, by, value):
    # elements = 


In [16]:
# 반복 수를 결정하는 경우
add_rep = 5

add_button = driver.find_element(By.CLASS_NAME, "fvwqf")
for _ in range(add_rep):
    # 더 보기 버튼 안보이면 break
    try:
        add_button.click()
        time.sleep(1)
    except:
        break

In [ ]:
# 목표 수집 댓글 수를 결정하는 경우
n = 200

add_button = driver.find_element(By.CLASS_NAME, "fvwqf")
while True:
    # 더 보기 버튼 안보이면 break
    try:
        add_button.click()
        time.sleep(1)
    except:
        # print("더 보기 버튼 안보임")
        break

    # 목표 리뷰 개수 도달하면 break
    reviews = driver.find_elements(By.CLASS_NAME, "zPfVt")
    if len(reviews) > n:
        # print("목표 리뷰 개수 도달")
        break


### 리뷰 추출하기
* 긴 글의 경우 내용 더보기 버튼이 활성화됨

In [27]:
test = reviews[0]

In [29]:
test.find_elements(By.CLASS_NAME, "rvCSr")

[]